In [9]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import MDAnalysis as md
import pyemma
from clustering import Clustering
import sklearn
import pandas as pd
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
import pandas.rpy.common as com
import matplotlib.pyplot as plt
from itertools import cycle
%matplotlib inline

SyntaxError: invalid syntax (mdamath.py, line 161)

In [5]:
topology = '/Volumes/RyanMdata/FUMP10/hairpin.pdb'
trajectory = '/Volumes/RyanMdata/FUMP10/Folding/weightedSims3200/weighted3200.dcd'

In [61]:
X = pyemma.coordinates.load([trajectory],top=topology)
#Brackets because you could provide a list of trajectories

In [44]:
#iKmeans -- basically does Kmeans 20 times
cl=Clustering()
data = cl.my_math.standardize(X)
[u, centroids, ite, dist_tmp, init_centroids] = cl.ik_means(data)
print('silhouette score: ' + str(sklearn.metrics.silhouette_score(data,u)))
print('Number of clusters: ' + str(max(u)))

silhouette score: 0.181008552653
Number of clusters: 41


In [43]:
#imwk_means -- ikmeans with a special weighting metric
[u, centroids, weights, ite, dist_tmp] = cl.imwk_means(data,2)
sklearn.metrics.silhouette_score(data,u)
print('silhouette score: ' + str(sklearn.metrics.silhouette_score(data,u)))
print('Number of clusters: ' + str(max(u)))

silhouette score: 0.120655
Number of clusters: 27


In [3]:
#Meanshift -- tries to discover blobs 
#bandwidth = estimate_bandwidth(X)
#ms = MeanShift(bandwidth=bandwidth, bin_seeding=False)
#ms.fit(X)
#labels = ms.labels_
#sklearn.metrics.silhouette_score(X,labels)

In [4]:
#DBscan
#data = StandardScaler().fit_transform(X)
#db = DBSCAN().fit(X)
#labels = db.labels_
#sklearn.metrics.silhouette_score(data,labels)

In [8]:
# Make a distance matrix for use in R
u = md.Universe(topology,trajectory)
numframes = len(u.trajectory)
for i in xrange(numframes):
    for j in xrange(i-1):
        d[i,j] = MDAnalysis.analysis.rms.rmsd( sel.coordinates(u.trajectory[i]), sel.coordinates(u.trajectory[j]), center=True)
        d[j,i] = d[i,j]

NameError: name 'md' is not defined

In [83]:
#QT from R
nr,nc = X.shape
Xr = ro.r.matrix(X, nrow=nr, ncol=nc)
ro.r.assign("X", Xr)
ro.r('library(flexclust)')
#ro.r('x <- matrix(10*runif(1000), ncol=2)')
ro.r('cl1 <- qtclust(X, radius=5)')
labels = np.array(ro.r('data.frame(cl=attributes(cl1)$cluster)'))
u = [x[0] for x in labels]
u = np.array(u)
sklearn.metrics.silhouette_score(data,u)

-0.036961824